# Homework 2

# Set up

## Installing packages

In [5]:
!pip install requests PyPDF2 gdown
!pip install 'markitdown[pdf]'
!pip install langchain_mcp_adapters langchain_google_genai langchain-openai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 5.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 1.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 68.1/68.1 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.4/15.4 MB 67.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.0/60.0 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.6/6.6 MB 99.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.4/63.4 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.1/17.1 MB 74.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 90.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.5/66.5 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 87.2/87.2 kB 5.3 MB/s eta 0:00:00


## Setup your API key

To run the following cell, your API key must be stored it in a Colab Secret named `VERTEX_API_KEY`.


1.   Look for the key icon on the left panel of your colab.
2.   Under `Name`, create `VERTEX_API_KEY`.
3. Copy your key to `Value`.

If you cannot use VERTEX_API_KEY, you can use deepseek models via `DEEPSEEK_API_KEY`. It does not affect your score.



In [6]:
from google.colab import userdata
DEEPSEEK_API_KEY = userdata.get('DEEPSEEK_API_KEY') # Retrieve Deepseek API key from Colab Secrets

# Download sample CVs

## Downloading sample_cv.pdf
The codes below download the sample CV


In [7]:
import os
import gdown

folder_id = "1adYKq7gSSczFP3iikfA8Er-HSZP6VM7D"
folder_url = f"https://drive.google.com/drive/folders/{folder_id}"

output_dir = "downloaded_cvs"
os.makedirs(output_dir, exist_ok=True)

gdown.download_folder(
    url=folder_url,
    output=output_dir,
    quiet=False,
    use_cookies=False
)

Retrieving folder contents


Processing file 1NR1RUKx4GyM7QOBxKXkfh4e8jUkxFCsp CV_1.pdf
Processing file 16lrd-uO8AAnCnv7UG9Rs_Nk6SUu0Iwbs CV_2.pdf
Processing file 15hVEuBan_EKhEty2aZBd6rcpDpP4o7Vr CV_3.pdf
Processing file 1Y2w_mAUEhg4vZBdvvR-0n3Jf2mKuGDRk CV_4.pdf
Processing file 1PLwkva-pdua6ZVvmLg9mxHeljq9D8C_C CV_5.pdf


Retrieving folder contents completed
Building directory structure
Building directory structure completed
Downloading...
From: https://drive.google.com/uc?id=1NR1RUKx4GyM7QOBxKXkfh4e8jUkxFCsp
To: /content/downloaded_cvs/CV_1.pdf
100%|██████████| 147k/147k [00:00<00:00, 45.4MB/s]
Downloading...
From: https://drive.google.com/uc?id=16lrd-uO8AAnCnv7UG9Rs_Nk6SUu0Iwbs
To: /content/downloaded_cvs/CV_2.pdf
100%|██████████| 75.1k/75.1k [00:00<00:00, 46.8MB/s]
Downloading...
From: https://drive.google.com/uc?id=15hVEuBan_EKhEty2aZBd6rcpDpP4o7Vr
To: /content/downloaded_cvs/CV_3.pdf
100%|██████████| 72.0k/72.0k [00:00<00:00, 41.0MB/s]
Downloading...
From: https://drive.google.com/uc?id=1Y2w_mAUEhg4vZBdvvR-0n3Jf2mKuGDRk
To: /content/downloaded_cvs/CV_4.pdf
100%|██████████| 73.3k/73.3k [00:00<00:00, 38.6MB/s]
Downloading...
From: https://drive.google.com/uc?id=1PLwkva-pdua6ZVvmLg9mxHeljq9D8C_C
To: /content/downloaded_cvs/CV_5.pdf
100%|██████████| 97.9k/97.9k [00:00<00:00, 62.1MB/s]
Download complete

['downloaded_cvs/CV_1.pdf',
 'downloaded_cvs/CV_2.pdf',
 'downloaded_cvs/CV_3.pdf',
 'downloaded_cvs/CV_4.pdf',
 'downloaded_cvs/CV_5.pdf']

In [8]:
# =====================================================
#  Load and display all CV PDFs in order
# =====================================================
import os
from markitdown import MarkItDown

cv_dir = "downloaded_cvs"

# Initialize MarkItDown
md = MarkItDown(enable_plugins=False)

# Collect and sort PDFs numerically
pdf_files = sorted(
    [f for f in os.listdir(cv_dir) if f.lower().endswith(".pdf")],
    key=lambda x: int("".join(filter(str.isdigit, x)))  # CV_1.pdf → 1
)

all_cvs = []

for pdf_name in pdf_files:
    pdf_path = os.path.join(cv_dir, pdf_name)
    result = md.convert(pdf_path)

    all_cvs.append({
        "file": pdf_name,
        "text": result.text_content
    })

    print("=" * 80)
    print(f"📄 {pdf_name}")
    print("=" * 80)
    print(result.text_content)
    print("\n\n")


📄 CV_1.pdf
|     |     |     |     | John         |           | Smith        |                   |     |     |
| --- | --- | --- | --- | ------------ | --------- | ------------ | ----------------- | --- | --- |
|     |     |     |     | Marketing    |           | Professional |                   |     |     |
|     |     |     |     | + Singapore, | Singapore |              | (cid:209) Kowloon |     |     |
Experience
|                |                  |     |          |                     |              |            |     | 2020 – | Present |
| -------------- | ---------------- | --- | -------- | ------------------- | ------------ | ---------- | --- | ------ | ------- |
| Engineer,      | ByteDance        |     |          |                     |              |            |     |        |         |
| • Worked       | in a fast-paced, |     | global   | technology          | environment. |            |     |        |         |
| • Collaborated | across           |     | teams to | sup

# Connect to our MCP server

Documentation about MCP: https://modelcontextprotocol.io/docs/getting-started/intro.

Using MCP servers in Langchain https://docs.langchain.com/oss/python/langchain/mcp.

## Check which tools that the MCP server provide

In [9]:
import asyncio
import json
from langchain_mcp_adapters.client import MultiServerMCPClient

client = MultiServerMCPClient({
    "social_graph": {
        "transport": "http",
        "url": "https://ftec5660.ngrok.app/mcp",
        "headers": {"ngrok-skip-browser-warning": "true"}
    }
})

mcp_tools = await client.get_tools()
for tool in mcp_tools:
    print(tool.name)
    print(tool.description)
    print(tool.args)
    print("\n\n------------------------------------------------------\n\n")

search_facebook_users
Search for Facebook users by display name (supports partial and fuzzy matching).

Args:
    q: Search query string (case-insensitive, matches any part of display name)
       Examples: "John", "john smith", "Smith"
    limit: Maximum number of results to return (default: 20, max: 20)
    fuzzy: Enable fuzzy matching if exact search returns no results (default: True)

Returns:
    List of user dictionaries, each containing:
    - id (int): Unique Facebook user ID for use with get_facebook_profile()
    - display_name (str): User's Facebook display name (may differ from legal name)
    - city (str): Current city of residence
    - country (str): Country of residence
    - match_type (str): "exact" or "fuzzy" (indicates search method used)
    
    Returns empty list [] if no matches found.

Example:
    search_facebook_users("Alex Chan", limit=5)
    → [{"id": 123, "display_name": "Alex Chan", "city": "Hong Kong", "country": "Hong Kong", "match_type": "exact"}]
    

## A simple agent using tools from the MCP server


In [10]:
import os
from langchain_core.tools import tool
from langchain_core.messages import HumanMessage
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_mcp_adapters.client import MultiServerMCPClient
from langchain_openai import ChatOpenAI



# ---------------------------
# 1. Define a local tool
# ---------------------------
@tool
def say_hello(name: str) -> str:
    """Say hello to a person by name."""
    return f"Hello, {name}! 👋"

# ---------------------------
# 2. Load MCP tools + merge
# ---------------------------
client = MultiServerMCPClient({
    "social_graph": {
        "transport": "http",
        "url": "https://ftec5660.ngrok.app/mcp",
        "headers": {"ngrok-skip-browser-warning": "true"}
    }
})

mcp_tools = await client.get_tools()
tools = mcp_tools + [say_hello]

# ---------------------------
# 3. Initialize Gemini (tool-enabled) or deepseek
# ---------------------------
# llm = ChatGoogleGenerativeAI(
#     model="gemini-2.0-flash",
#     google_api_key=GEMINI_VERTEX_API_KEY,
#     temperature=0,
# )

# DEEPSEEK_API_KEY = userdata.get("DEEPSEEK_API_KEY") # This line is moved to ueILmCPHci9v
llm = ChatOpenAI(
    model="deepseek-chat",          # or "deepseek-reasoner"
    api_key=DEEPSEEK_API_KEY,
    base_url="https://aihubmix.com/v1",
    temperature=0,
)

llm_with_tools = llm.bind_tools(tools)

# ---------------------------
# 4. Single-step invocation
# ---------------------------
query = "Say hello to Bao using tool, then search for someone named Alice on Facebook."

response = llm_with_tools.invoke([
    HumanMessage(content=query)
])

print(response)

content="I'll help you with that! First, let me say hello to Bao, and then search for Alice on Facebook." additional_kwargs={'refusal': None} response_metadata={'token_usage': {'completion_tokens': 68, 'prompt_tokens': 2940, 'total_tokens': 3008, 'completion_tokens_details': None, 'prompt_tokens_details': {'audio_tokens': None, 'cached_tokens': 2880}}, 'model_provider': 'openai', 'model_name': 'deepseek-chat', 'system_fingerprint': 'fp_eaab8d114b_prod0820_fp8_kvcache', 'id': 'fdc5959e-71a4-47d5-8c7b-c95e2b1027ea', 'finish_reason': 'tool_calls', 'logprobs': None} id='lc_run--019c93ae-61fb-77a2-95f5-ea63f22962e0-0' tool_calls=[{'name': 'say_hello', 'args': {'name': 'Bao'}, 'id': 'call_00_7htQJNxSLcUhX7bDaPNW4OMD', 'type': 'tool_call'}] invalid_tool_calls=[] usage_metadata={'input_tokens': 2940, 'output_tokens': 68, 'total_tokens': 3008, 'input_token_details': {'cache_read': 2880}, 'output_token_details': {}}


In [14]:
# Test 1: Search Facebook users (exact match)
R1 = await tools[0].ainvoke({'q': "Alex Chan", 'limit': 5})
print("Test 1 - search_facebook_users (exact match):")
print(R1)

# Test 2: Search Facebook users (fuzzy match with typo)
R2 = await tools[0].ainvoke({'q': 'Alx Chn', 'limit': 5, 'fuzzy': True})
print("Test 2 - search_facebook_users (fuzzy match):")
print(R2)

# Test 3: Get Facebook profile
R3 = await tools[1].ainvoke({'user_id': 123})
print("Test 3 - get_facebook_profile:")
print(R3)

# Test 4: Get Facebook mutual friends
R4 = await tools[2].ainvoke({'user_id_1': 123, 'user_id_2': 456})
print("Test 4 - get_facebook_mutual_friends:")
print(R4)

# Test 5: Search LinkedIn people (exact match)
R5 = await tools[3].ainvoke({'q': "Python", 'location': "Hong Kong", 'limit': 5})
print("Test 5 - search_linkedin_people (exact match):")
print(R5)

# Test 6: Search LinkedIn people (fuzzy match with typo)
R6 = await tools[3].ainvoke({'q': "Python", 'location': "Hong Kong", 'limit': 5, 'fuzzy': True})
print("Test 6 - search_linkedin_people (fuzzy match):")
print(R6)

# Test 7: Get LinkedIn profile
R7 = await tools[4].ainvoke({'person_id': 456})
print("Test 7 - get_linkedin_profile:")
print(R7)

# Test 8: Get LinkedIn interactions
R8 = await tools[5].ainvoke({'person_id': 456})
print("Test 8 - get_linkedin_interactions:")
print(R8)


Test 1 - search_facebook_users (exact match):
[{'type': 'text', 'text': '[{"id":3,"display_name":"Alex Chan","city":"Hyderabad","country":"India","match_type":"exact"},{"id":41,"display_name":"Alex Chan","city":"Shanghai","country":"China","match_type":"exact"},{"id":79,"display_name":"Alex Chan","city":"London","country":"UK","match_type":"exact"},{"id":106,"display_name":"Alex Chan","city":"Ho Chi Minh City","country":"Vietnam","match_type":"exact"},{"id":117,"display_name":"Alex Chan","city":"Melbourne","country":"Australia","match_type":"exact"}]', 'id': 'lc_cb28f586-b710-4f95-bf6e-643fed0cdc1f'}]
Test 2 - search_facebook_users (fuzzy match):
[{'type': 'text', 'text': '[{"id":3,"display_name":"Alex Chan","city":"Hyderabad","country":"India","match_type":"fuzzy"},{"id":41,"display_name":"Alex Chan","city":"Shanghai","country":"China","match_type":"fuzzy"},{"id":79,"display_name":"Alex Chan","city":"London","country":"UK","match_type":"fuzzy"},{"id":106,"display_name":"Alex Chan","ci

# Evaluation code

In the test phase, you will be given 5 CV files with fixed names:

    CV_1.pdf, CV_2.pdf, CV_3.pdf, CV_4.pdf, CV_5.pdf

Your system must process these CVs and output a list of 5 scores,
one score per CV, in the same order:

    scores = [s1, s2, s3, s4, s5]

Each score must be a float in the range [0, 1], representing the
reliability or confidence that the CV is valid (or meets the task criteria).

The ground-truth labels are binary:

    groundtruth = [0 or 1, ..., 0 or 1]

Each CV is evaluated independently using a threshold of 0.5:

- If score > 0.5 and groundtruth == 1 → Full credit
- If score ≤ 0.5 and groundtruth == 0 → Full credit
- Otherwise → No credit

In other words, 0.5 is the decision threshold.

- Each CV contributes equally.
- Final score = (number of correct decisions) / 5


In [15]:
# =====================================================
#  Evaluation code
# =====================================================

def evaluate(scores, groundtruth, threshold=0.5):
    """
    scores: list of floats in [0, 1], length = 5
    groundtruth: list of ints (0 or 1), length = 5
    """
    assert len(scores) == 5
    assert len(groundtruth) == 5

    correct = 0
    decisions = []

    for s, gt in zip(scores, groundtruth):
        pred = 1 if s > threshold else 0
        decisions.append(pred)
        if pred == gt:
            correct += 1

    final_score = correct / len(scores)

    return {
        "decisions": decisions,
        "correct": correct,
        "total": len(scores),
        "final_score": final_score
    }


In [16]:
import json
import re

scores = []

for cv_data in all_cvs:
    cv_text = cv_data['text']
    query = f"Please evaluate the following CV for its general validity and completeness. Provide a confidence score between 0.0 and 1.0. Respond only with a JSON object: {{'score': float_value}}.\n\nCV Text:\n{cv_text}"

    # Use the base LLM instance without tools for this specific scoring query
    response = llm.invoke([
        HumanMessage(content=query)
    ])

    # Attempt to parse the score from the LLM's response
    try:
        # Extract JSON string from potential markdown block
        json_match = re.search(r"```json\n([\s\S]*?)\n```", response.content)
        if json_match:
            json_string = json_match.group(1)
        else:
            json_string = response.content # Assume it's direct JSON if no markdown block

        score_dict = json.loads(json_string)
        score = float(score_dict.get('score', 0.0)) # Default to 0 if score not found
        # Ensure score is within [0, 1]
        score = max(0.0, min(1.0, score))
    except (json.JSONDecodeError, ValueError) as e:
        print(f"Could not parse score for {cv_data['file']}, defaulting to 0.5. Error: {e}. Response: {response.content}")
        score = 0.5 # Default score if parsing fails

    scores.append(score)

print(f"Generated scores: {scores}")

groundtruth = [1, 1, 1, 0, 0] # Do not modify

result = evaluate(scores, groundtruth)
print(result)

Generated scores: [0.4, 0.65, 0.3, 0.2, 0.2]
{'decisions': [0, 1, 0, 0, 0], 'correct': 3, 'total': 5, 'final_score': 0.6}
